In [1]:
import numpy as np
from sklearn.utils import shuffle
from utils.utils import load_data_tensorflow
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import *
from sklearn.utils import shuffle
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from utils.utils import load_data_tensorflow
from tensorflow.keras import datasets, layers, models, callbacks
from keras.losses import SparseCategoricalCrossentropy

In [2]:
img = 150
names = ['O', 'R']
encode_name = {name: i for i, name in enumerate(names)}
epochs = 2
batch_size = 25

In [3]:
def generate_model():
    model = models.Sequential(name='CNN-WASTE-CLASSIFICATION')
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img, img, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10))
    return model

## Etapa de Pré-Processamento

In [4]:
(train_images, train_labels), (test_images, test_labels) = load_data_tensorflow()

train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

train_images, test_images = train_images / 255, test_images / 255
train_images, train_labels = shuffle(train_images, train_labels)

100%|██████████| 2/2 [02:15<00:00, 67.68s/it]


## Variação de Parâmetros dos Otimizadores

In [5]:
# Geração de Modelo
model = generate_model()
model.summary()
model.save('model.h5')
callback = callbacks.EarlyStopping(monitor='loss', patience=3)

Model: "CNN-WASTE-CLASSIFICATION"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 73984)             0         
_________________________________________________________________
dense (Dense)                (None, 64)   

In [6]:
def optimizer(opt_name, loss, callbacks):
    model.compile(optimizer=opt_name,
              loss=loss,
              metrics=['accuracy'])

    model.load_weights('model.h5')

    history = model.fit(train_images, 
        train_labels, epochs=30,
        callbacks=callbacks,
        validation_data=(test_images, test_labels))

    y_pred = model.evaluate(test_images, test_labels, batch_size=128)
    print("test loss, test acc:", y_pred)
    print(f"{len(history.history['loss'])} - epochs utilizadas")

## Primeiro Otimizador

In [7]:
for x in range(5):
    print(x)
    optimizer('adam', 'binary_crossentropy', callback)

0
Epoch 1/30
438/438 [==============================] - 259s 588ms/step - loss: 7.6450 - accuracy: 0.0016 - val_loss: 7.6510 - val_accuracy: 0.0000e+00
Epoch 2/30
438/438 [==============================] - 270s 616ms/step - loss: 7.6509 - accuracy: 0.0000e+00 - val_loss: 7.6510 - val_accuracy: 0.0000e+00
Epoch 3/30
438/438 [==============================] - 256s 584ms/step - loss: 7.6509 - accuracy: 0.0000e+00 - val_loss: 7.6510 - val_accuracy: 0.0000e+00
Epoch 4/30
16/16 [==============================] - 10s 634ms/step - loss: 7.6510 - accuracy: 0.0000e+00
test loss, test acc: [7.65097713470459, 0.0]
4 - epochs utilizadas
1
Epoch 1/30
438/438 [==============================] - 226s 514ms/step - loss: 7.6533 - accuracy: 0.0014 - val_loss: 7.6598 - val_accuracy: 0.0000e+00
Epoch 2/30
438/438 [==============================] - 226s 516ms/step - loss: 7.6598 - accuracy: 0.0000e+00 - val_loss: 7.6598 - val_accuracy: 0.0000e+00
Epoch 3/30
438/438 [==============================] - 220s 502

## Segundo Otimizador

In [8]:
# Otimizador executado no primeiro tp
for x in range(5):
    optimizer(Adam(learning_rate=0.0001), 'binary_crossentropy', callback)

Epoch 1/30
438/438 [==============================] - 222s 506ms/step - loss: 0.9815 - accuracy: 0.0838 - val_loss: 0.7561 - val_accuracy: 0.0355
Epoch 2/30
438/438 [==============================] - 211s 483ms/step - loss: 0.5743 - accuracy: 0.0279 - val_loss: 0.5458 - val_accuracy: 0.0715
Epoch 3/30
438/438 [==============================] - 207s 473ms/step - loss: 0.5260 - accuracy: 0.0959 - val_loss: 0.4908 - val_accuracy: 0.0075
Epoch 4/30
438/438 [==============================] - 222s 506ms/step - loss: 0.6799 - accuracy: 0.0461 - val_loss: 0.6100 - val_accuracy: 0.0400
Epoch 5/30
438/438 [==============================] - 217s 495ms/step - loss: 0.5093 - accuracy: 0.1316 - val_loss: 0.4423 - val_accuracy: 0.2775
Epoch 6/30
438/438 [==============================] - 219s 501ms/step - loss: 0.4844 - accuracy: 0.1862 - val_loss: 0.4300 - val_accuracy: 0.3995
Epoch 7/30
438/438 [==============================] - 214s 489ms/step - loss: 0.4740 - accuracy: 0.2903 - val_loss: 0.4537 -

## Terceiro Otimizador

In [9]:
for x in range(5):
    print(x)
    optimizer(SGD(learning_rate=0.1), SparseCategoricalCrossentropy(from_logits=True), callback)

0
Epoch 1/30
438/438 [==============================] - 201s 455ms/step - loss: 0.9212 - accuracy: 0.5016 - val_loss: 0.6971 - val_accuracy: 0.5000
Epoch 2/30
438/438 [==============================] - 207s 472ms/step - loss: 0.6992 - accuracy: 0.5003 - val_loss: 0.6967 - val_accuracy: 0.5000
Epoch 3/30
438/438 [==============================] - 205s 468ms/step - loss: 0.6985 - accuracy: 0.5007 - val_loss: 0.6976 - val_accuracy: 0.5000
Epoch 4/30
438/438 [==============================] - 202s 462ms/step - loss: 0.6973 - accuracy: 0.5089 - val_loss: 0.6953 - val_accuracy: 0.5000
Epoch 5/30
438/438 [==============================] - 206s 471ms/step - loss: 0.6978 - accuracy: 0.5034 - val_loss: 0.6960 - val_accuracy: 0.5000
Epoch 6/30
438/438 [==============================] - 203s 463ms/step - loss: 0.6979 - accuracy: 0.4996 - val_loss: 0.6942 - val_accuracy: 0.5000
Epoch 7/30
438/438 [==============================] - 202s 461ms/step - loss: 0.6969 - accuracy: 0.5060 - val_loss: 0.6985

In [10]:
# for x in range(5):
for x in range(5):
    print(x)
    optimizer('adam', SparseCategoricalCrossentropy(from_logits=True), callback)

0
Epoch 1/30
438/438 [==============================] - 216s 492ms/step - loss: 0.4902 - accuracy: 0.7759 - val_loss: 0.4106 - val_accuracy: 0.8190
Epoch 2/30
438/438 [==============================] - 217s 495ms/step - loss: 0.4026 - accuracy: 0.8217 - val_loss: 0.3376 - val_accuracy: 0.8655
Epoch 3/30
438/438 [==============================] - 213s 486ms/step - loss: 0.3503 - accuracy: 0.8492 - val_loss: 0.3074 - val_accuracy: 0.8715
Epoch 4/30
438/438 [==============================] - 216s 492ms/step - loss: 0.2926 - accuracy: 0.8798 - val_loss: 0.3248 - val_accuracy: 0.8705
Epoch 5/30
438/438 [==============================] - 217s 495ms/step - loss: 0.2201 - accuracy: 0.9101 - val_loss: 0.3809 - val_accuracy: 0.8530
Epoch 6/30
438/438 [==============================] - 221s 506ms/step - loss: 0.1338 - accuracy: 0.9502 - val_loss: 0.4741 - val_accuracy: 0.8545
Epoch 7/30
438/438 [==============================] - 216s 492ms/step - loss: 0.0739 - accuracy: 0.9738 - val_loss: 0.5826